# Welcome to the Week 3 Capstone
## Notebook number 2
**This projects consists of a few steps**
1. Create notebook and github repository as seen in assignment one
2. Scrape the provided wikipedia page 
3. Transform data into a *pandas* Dataframe


In [1]:
#all of my imports so I have a dedicated cell to just imports 
import geocoder
import wikipedia
from bs4 import BeautifulSoup
import requests as rq
import numpy as np
import pandas as pd

I know that you could use pandas to read the table into a pandas dataframe. 
The articles I read and videos I watched to supplement my python learning went over BeautifulSoup package. I went with that because it helpped me apply what I learned better to what I was trying to achieve in this project. If you would like a reference list on articles and videos please let me know. 

### Scrape Wikipedia Page/Postal Codes

In [2]:
def get_geocoder(postal_code):
    # initialize variable
    coordinates = None

    #loop until coordinates are received 
    while(coordinates is None):
        goog = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        coordinates = goog.latlng
    
    latitude = coordinates[0]
    longitude = coordinates[1]
    return (latitude,longitude)

In [3]:
postal_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = rq.get(postal_url).text

cand_html = BeautifulSoup(source, 'xml')
cand_html = cand_html.find('table')


#### Convert the html

In [4]:
#Table has 3 (three) columns Postal Code, Borough, Neighborhood
col_title = ["Postal Code", "Borough", "Neighborhood"]
cand_df = pd.DataFrame(columns = col_title)

#Search all the postcode, borough, neighborhood
for tr_cell in cand_html.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if (len(row_data) == 3):
        cand_df.loc[len(cand_df)] = row_data
        
cand_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Cleansing the Data
- Looking at the main table on the wikipedia page you will find some postal codes that are not assigned to any borough/neighborhood
    - These postal codes exist in the post system but do not have any location value
- We are going to remove those from the table since we are only interested in physical locations (i.e Toronto) and not the complexities of the Canadian postal system. 
    - My research has lead me to assume these not assigned postal codes are for areas that could be developed in the future but as of now have no mail. 
    
- Some neighborhoods share a postal code, that is okay, they are seperated by comma

In [5]:
#clean/Delete Not assigned cells
cand_df['Borough'].replace("Not assigned", np.nan, inplace = True)
cand_df["Neighborhood"].replace("Not assigned", np.nan, inplace=True)
cand_df.dropna(subset = ['Borough'], axis = 0, inplace = True)
cand_df["Neighborhood"].fillna(cand_df['Borough'], inplace = True)

#Check work
cand_df.head(8)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills


In [6]:
geo = pd.read_csv('Geospatial_Coordinates.csv')

final_df = pd.merge(cand_df,geo)

final_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
